In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import seaborn as sns
from glob import glob
import os.path
from matplotlib.pyplot import subplots
import numpy as np

# Retrieve activity data from ChEMBL

In [53]:
import urllib.parse
import urllib.request

url = 'https://www.uniprot.org/uploadlists/'

params = {
'from': 'ACC+ID',
'to': 'CHEMBL_ID',
'format': 'tab',
'query': 'P49892'
}

data = urllib.parse.urlencode(params)
data = data.encode('utf-8')
req = urllib.request.Request(url, data)
with urllib.request.urlopen(req) as f:
   response = f.read()
print(response.decode('utf-8'))

From	To
P49892	CHEMBL2114



In [54]:
from chembl_webresource_client.new_client import new_client

activity = new_client.activity

In [56]:
activity_data = activity.filter(target_chembl_id='CHEMBL2114')

In [63]:
activity_data = activity.filter(target_components__accession='P49892')

In [64]:
len(activity_data)

18635916

In [60]:
activity_data[0]

{'activity_comment': None,
 'activity_id': 753534,
 'activity_properties': [],
 'assay_chembl_id': 'CHEMBL640604',
 'assay_description': 'Displacement of [125I]ABA from adenosine A1 receptor of chick cerebellar membrane',
 'assay_type': 'B',
 'assay_variant_accession': None,
 'assay_variant_mutation': None,
 'bao_endpoint': 'BAO_0000190',
 'bao_format': 'BAO_0000019',
 'bao_label': 'assay format',
 'canonical_smiles': 'OC[C@H]1OC(n2cnc3c(Nc4cccc(F)c4)ncnc32)[C@H](O)[C@@H]1O',
 'data_validity_comment': None,
 'data_validity_description': None,
 'document_chembl_id': 'CHEMBL1123754',
 'document_journal': 'J. Med. Chem.',
 'document_year': 1987,
 'ligand_efficiency': {'bei': '22.10',
  'le': '0.42',
  'lle': '7.67',
  'sei': '6.36'},
 'molecule_chembl_id': 'CHEMBL607777',
 'molecule_pref_name': None,
 'parent_molecule_chembl_id': 'CHEMBL607777',
 'pchembl_value': '7.99',
 'potential_duplicate': False,
 'qudt_units': 'http://www.openphacts.org/units/Nanomolar',
 'record_id': 345225,
 'rela

In [2]:
from chembl_webresource_client.new_client import new_client

available_resources = [resource for resource in dir(new_client) if not resource.startswith('_')]
print(available_resources)

['activity', 'activity_supplementary_data_by_activity', 'assay', 'assay_class', 'atc_class', 'binding_site', 'biotherapeutic', 'cell_line', 'chembl_id_lookup', 'compound_record', 'compound_structural_alert', 'description', 'document', 'document_similarity', 'drug', 'drug_indication', 'drug_warning', 'go_slim', 'image', 'mechanism', 'metabolism', 'molecule', 'molecule_form', 'official', 'organism', 'protein_class', 'similarity', 'source', 'substructure', 'target', 'target_component', 'target_relation', 'tissue', 'xref_source']


In [ ]:
assays.bioactivities('CHEMBL1217643')

In [3]:
target = new_client.target

In [29]:
target_data_0 = target.filter(target_chembl_id='CHEMBL2114')

In [30]:
target_data_1 = target.filter(chembl_id='CHEMBL2114')

In [65]:
target_data_2 = target.filter(target_components__accession='P49892')

In [67]:
target_data_2[0]['target_chembl_id']

'CHEMBL2114'

In [68]:
len(target_data_2)

1

In [38]:
target_data_0[0] == target_data_1[0]

True

In [42]:
target_data_0[0] == target_data_2[0]

True

In [ ]:
molecule = new_client.molecule
mols = molecule.filter(pref_name__iexact='aspirin')
mols

In [ ]:
import urllib
import json
import re

In [ ]:
def get_ligands(accession):

    data = []

    target_data = json.loads(urllib.urlopen("https://www.ebi.ac.uk/chemblws/targets/uniprot/%s.json" %
                                             accession).read())
    bioactivity_data = json.loads(urllib.urlopen("https://www.ebi.ac.uk/chemblws/targets/%s/bioactivities.json" %
                                                  target_data['target']['chemblId']).read())

    for bioactivity in bioactivity_data['bioactivities']:
        
        if not is_number(bioactivity["value"]):
            continue

        row = {}
        
        for key in ["bioactivity_type", "operator", "value", "units", "target_chemblid",
                    "ingredient_cmpd_chemblid"]:
            row[key] = bioactivity[key]
        data.append(row)

    return data

In [ ]:
get_ligands('P49892')

In [ ]:
names = []

all_data = pd.DataFrame(columns=["name", "uniprot_name", "target_chemblid", "bioactivity_type",
                                 "operator", "value", "units", "ingredient_cmpd_chemblid"])

for i in glob("../data/targets/*"):
    name = i.split("/")[-1][:-4]
    print(name)
    names.append(name)
    f = open(i)
    for line in f:
        tmp = line.split("\t")
        accession = tmp[0]
        uniprot_name = tmp[1]
        try:
            tmp_data = get_ligands(accession)
            if tmp_data:
                tmp_pd = pd.DataFrame(data=tmp_data, columns=["name", "uniprot_name", "target_chemblid",
                                                                  "units", "bioactivity_type", "operator",
                                                                  "value", "ingredient_cmpd_chemblid"])
                tmp_pd["name"] = name
                tmp_pd["uniprot_name"] = uniprot_name

                all_data = pd.concat([all_data, tmp_pd], ignore_index=True)
        except:
            continue

    f.close()
all_data.to_csv("../data/raw_data.csv")

In [ ]:
all_data